In [1]:
from datetime import datetime
import os
#  import time
#  from multiprocess import Pool
import numpy as np
import iris
import iris.coord_categorisation as icc
import math
import matplotlib.pyplot as plt
from pyts.decomposition import SSA

In [2]:
# SSA options
window_size = 365
grouping = 10

In [3]:
# specify paths
out_script = '/home/bschmidt/scripts/detrending/output/gmt.out'
source_path = '/home/bschmidt/data/test_data_tas.nc4'
dest_path = '/home/bschmidt/data/test_gmt.nc'

# Get jobs starting time
STIME = datetime.now()
with open(out_script, 'w') as out:
    out.write('Job started at: ' + str(STIME) + '\n')
print('Job started at: ' + str(STIME))

# load data and add auxiliary variable "doy"
data = iris.load_cube(source_path)
icc.add_day_of_year(data, 'time')

#  Let iris figure out cell boundaries and calculate
#  global mean temperatures weighted by cell area
data.coord('latitude').guess_bounds()
data.coord('longitude').guess_bounds()
grid_areas = iris.analysis.cartography.area_weights(data)
# .collapsed method applieas a function to a grid and outputs
col = data.collapsed(['longitude', 'latitude'],
                                iris.analysis.MEAN,
                                weights=grid_areas)
del data

Job started at: 2019-03-12 15:01:01.919503


/home/bschmidt/.programs/anaconda3/envs/detrending/lib/python3.7/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")


In [4]:
col = np.array(col.data[::10], ndmin=2)
print(col)

[[285.49600647 285.63576058 285.7278155  ... 286.7350526  286.52409364
  286.45364832]]


In [ ]:
fft = np.fft.fft(col.data, axis=0)
freq = np.fft.fftfreq(col.shape[-1])
print(fft.imag)
x = freq[:math.floor(len(col.data)/2)]
for i in range(len(x)):
    if x[i] > 0.0001: # cut off all frequencies higher than 0.005
        fft[i] = 0.0
        fft[math.floor(len(col.data)/2) + i] = 0.0
inverse = np.fft.ifft(fft)

In [5]:
ssa = SSA(window_size, grouping)
X_ssa = ssa.fit_transform(col)

In [ ]:
# Show the results for the first time series
plt.figure(figsize=(12, 8))
plt.plot(inverse, '--', label='low-passed')
#plt.plot()
#plt.plot(fft, 'o-', label='Original')
#plt.plot(freq, fft.real, freq, fft.imag)
#plt.ylim((0, 500))
#plt.plot(X_ssa[0, 0], 'o--', label='SSA')
plt.legend(loc='best', fontsize=14)
plt.show()